In [1]:
from pyspark.sql import SparkSession
from delta import *

import subprocess
import json
import os

In [2]:
from uuid import uuid4

In [3]:
session_id = str(uuid4())

In [ ]:
builder = SparkSession.builder.appName(session_id) \
    .master("spark://spark-master:7077")\
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.4.0")\
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

spark = configure_spark_with_delta_pip(builder).getOrCreate()    
spark.sparkContext.setLogLevel("WARN")

In [5]:
from confluent_kafka.admin import AdminClient, NewTopic
from confluent_kafka import Producer, Consumer, KafkaError
from confluent_kafka.schema_registry import SchemaRegistryClient, Schema
from confluent_kafka import avro
import pyspark.sql.functions as pyspark_f
import pyspark.sql.types as pyspark_dt

# Create Topic & Schema Registry

In [6]:
topic_name = 'sample_topic'
kafka_config = {
    'bootstrap.servers': 'kafka-1:9092,kafka-2:9092,kafka-3:9092',
}

schema_registry_cofig =  {'url' : 'http://schema-registry:8085'} # URL for Schema Registry
schema_registry_subject = 'schema_registry_subject_example' 

In [55]:
# Create Kafka topic
admin_client = AdminClient(kafka_config)
new_topic = NewTopic(topic_name, num_partitions=2, replication_factor=2)

# Create the topic
admin_client.create_topics([new_topic])
print(f"Kafka topic {topic_name} created.")

Kafka topic sample_topic created.


In [62]:
producer = Producer(kafka_config)

In [64]:
# Produce messages
for i in range(10):
    key = str(i)
    value = {"name": f"User_{i}", "age": 20 + i}
    value_json = json.dumps(value)  # Convert dictionary to JSON string
    producer.produce(topic=topic_name, key=key, value=value_json)
    print(f"Produced Avro message: {value}")
producer.flush()

Produced Avro message: {'name': 'User_0', 'age': 20}
Produced Avro message: {'name': 'User_1', 'age': 21}
Produced Avro message: {'name': 'User_2', 'age': 22}
Produced Avro message: {'name': 'User_3', 'age': 23}
Produced Avro message: {'name': 'User_4', 'age': 24}
Produced Avro message: {'name': 'User_5', 'age': 25}
Produced Avro message: {'name': 'User_6', 'age': 26}
Produced Avro message: {'name': 'User_7', 'age': 27}
Produced Avro message: {'name': 'User_8', 'age': 28}
Produced Avro message: {'name': 'User_9', 'age': 29}


0

In [7]:
df = spark \
    .readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", kafka_config['bootstrap.servers']) \
    .option("subscribe", topic_name) \
    .option("startingOffsets", "earliest") \
    .load()


AnalysisException: Failed to find data source: kafka. Please deploy the application as per the deployment section of Structured Streaming + Kafka Integration Guide.